# Deep Learning Tests

## 1. Linear Regression
https://d2l.ai/chapter_linear-networks/linear-regression-scratch.html

### 1.1. Linear regression from scratch with NumPy

In [17]:
import numpy as np
import random

# Define the true weights and bias of the model
w_true = np.array([2, -3.4])
b_true = 4.2

# Generate inputs, sampled from a standard normal distribution
number_examples = 1000
number_features = len(w_true)
X = np.random.default_rng().normal(0, 1, (number_examples, number_features))

# Derive the outputs, with some noise
y = np.matmul(X, w_true)+b_true+np.random.default_rng().normal(0, 0.01, number_examples)

# Define the parameters for the training
lr = 0.03
number_epochs = 3
batch_size = 10

# Initialize the weights and bias to recover
w = np.random.default_rng().normal(0, 1, number_features)
b = 0

# Initialize an array for the mean loss over the minibatches of every epoch
epoch_loss = np.zeros(number_epochs)

# Loop over the epochs
for i in range(number_epochs):
    
    # Generate the indices for all the examples and shuffle them
    example_indices =  np.arange(number_examples)
    random.shuffle(example_indices)
    
    # Initialize a list for the mean loss over the examples of every minbatch
    batch_loss = []
    
    # Loop over the examples in batches
    for j in np.arange(0, number_examples, batch_size):
        
        # Get the indices of the (randomized) examples for one minibatch
        batch_indices = example_indices[j:min(j+batch_size, number_examples)]
        
        # Get the inputs and outputs for the current minibatch
        X_batch = X[batch_indices, :]
        y_batch = y[batch_indices]
        
        # Compute the predicted outputs
        y_hat = np.matmul(X_batch, w) + b
        
        # Compute the loss between the predicted and true outputs
        l = np.mean(0.5*np.power(y_hat-y_batch, 2))
        
        # Save the mean loss for the current minibatch
        batch_loss.append(l)
        
        # Update the weights and bias using stochastic gradient descent
        w = w - lr*np.mean(X_batch*(y_hat-y_batch)[:, np.newaxis], axis=0)
        b = b - lr*np.mean(y_hat-y_batch, axis=0)
        
    # Update the mean loss for the current epoch
    epoch_loss[i] = np.mean(batch_loss)
    
    # Print the progress
    print(f'{i+1}/{number_epochs}: {epoch_loss[i]}')

1/3: 34.28472425766261
2/3: 0.12353325147705162
3/3: 0.0008822978982950066


### 1.2. Linear regression from scratch with PyTorch

In [18]:
import torch
import random

# Define the true weights and bias of the model
w_true = torch.tensor([2, -3.4])
b_true = 4.2

# Generate inputs, sampled from a standard normal distribution
number_examples = 1000
number_features = len(w_true)
X = torch.normal(0, 1, (number_examples, number_features))

# Derive the outputs, with some noise
y = torch.matmul(X, w_true)+b_true+torch.normal(0, 0.01, [number_examples]) # ?

# Define the parameters for the training
lr = 0.03
number_epochs = 3
batch_size = 10

# Initialize the weights and bias to recover, requiring the gradients to be computed
w = torch.normal(0, 1, [number_features], requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# Initialize an array for the mean loss over the minibatches of every epoch
epoch_loss = torch.zeros(number_epochs)

# Define a function to read the dataset in random batches
def batch(X, y, batch_size):
    
    # Generate the indices for all the examples and shuffle them
    number_examples = X.shape[0]
    example_indices = list(range(number_examples))
    random.shuffle(example_indices)
    
    # Loop over the examples in batches
    for i in range(0, number_examples, batch_size):
        
        # Get the indices of the (randomized) examples for one minibatch
        batch_indices = example_indices[i:min(i+batch_size, number_examples)]
        
        # Return the input and output minibatches and continue the iteration in the function
        yield X[batch_indices], y[batch_indices]
        
# Loop over the epochs
for i in range(number_epochs):
    
    # Loop over the examples in batches
    for X_batch, y_batch in batch(X, y, batch_size):
        
        # Compute the predicted outputs
        y_hat = torch.matmul(X_batch, w) + b
        
        # Compute the loss between the predicted and true outputs
        l = 0.5*(y_hat-y_batch)**2
        
        # Compute the gradient on l with respect to w and b
        l.sum().backward() # ?
        
#         # Save the mean loss for the current minibatch
#         batch_loss.append(l)
        
#         # Update the weights and bias using stochastic gradient descent
#         w = w - lr*np.mean(X_batch*(y_hat-y_batch)[:, np.newaxis], axis=0)
#         b = b - lr*np.mean(y_hat-y_batch, axis=0)

In [7]:
y_hat.shape

torch.Size([10])

In [8]:
y_batch.shape

torch.Size([10])

In [23]:
x.grad.zero_()
y = x * x

In [25]:
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [26]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [27]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [28]:
d

tensor(1802.0082, grad_fn=<MulBackward0>)

In [30]:
a.grad

tensor(8192.)

In [31]:
d/a

tensor(8192., grad_fn=<DivBackward0>)